In [ ]:
import geopandas
import shapely
import pathlib

In [ ]:
r_drive = pathlib.Path(r"R:\National\Datasets")
tide_points = geopandas.read_file(r_drive / "New_Zealand_Water_Modelling" / "NZTide_Data" / "NZTide_GIS" / "depth_surface" / "new_points.shp")
tide_points=tide_points.to_crs(2193)
local_drive = pathlib.Path(r"c:\Local\data\DEMs\tide model")
tide_points.to_file(local_drive / "msl_tide_model_points.gpkg")

# Load in datasets from R-drive

In [ ]:
r_drive = pathlib.Path(r"R:\National\Datasets")
tide_points = geopandas.read_file(r_drive / "New_Zealand_Water_Modelling" / "NZTide_Data" / "NZTide_GIS" / "depth_surface" / "new_points.shp")
tiles = geopandas.read_file(r_drive / "DigitalElevationModel" / "HydroDEM" / "National_8m" / "geofabrics-tiles-2024-03-01" / "tiles.gpkg")
convertor = geopandas.read_file(r_drive / "DigitalElevationModel" / "NZVD2016_conversion" / "NZ_NZVD2016_MSL_conversion.gpkg")

# Clip to dataset around NZ coast and save

In [ ]:
crs = 2193
distance = 20000  # 20 km offshore
tide_points=tide_points.to_crs(2193)
tide_points = tide_points.clip(tiles.dissolve().buffer(distance))

## Save a local copy

In [ ]:
local_drive = pathlib.Path(r"c:\Local\data\DEMs\tide model")
tide_points.to_file(local_drive / "msl_clipped_tide_model_points.gpkg")

# Convert offshore points to NZVD2016

In [ ]:
def tide_to_nzvd2016(row, vertical_datum_conversion):
    index = vertical_datum_conversion.sindex.nearest(row.geometry, return_all=False)[1][0]
    nzvd2016_to_msl = vertical_datum_conversion.iloc[index]["NZVD16_MSL"]
    return row["Z"] - nzvd2016_to_msl
tide_points["Z"] = tide_points.apply(lambda row: tide_to_nzvd2016(row, convertor), axis=1)
tide_points["geometry"] = tide_points.apply(lambda row: shapely.geometry.Point([row["geometry"].x, row["geometry"].y, row["Z"]]), axis=1)

## Save a local copy

In [ ]:
local_drive = pathlib.Path(r"c:\Local\data\DEMs\tide model")
tide_points.to_file(local_drive / "nzvd2016_clipped_tide_model_points.gpkg")

## Enforce a max value of mean low water spring

In [ ]:
mean_low_water_spring = -1.2
tide_points.loc[tide_points['Z'] > mean_low_water_spring, 'Z'] = mean_low_water_spring
tide_points["geometry"] = tide_points.apply(lambda row: shapely.geometry.Point([row["geometry"].x, row["geometry"].y, row["Z"]]), axis=1)

## Save a local copy

In [ ]:
local_drive = pathlib.Path(r"c:\Local\data\DEMs\tide model")
tide_points.to_file(local_drive / "nzvd2016_clipped_tide_model_points_max_set_to_mlws.gpkg")

## Enforce max value to mean sea level & Save

In [ ]:
mean_sea_level = -0.2
tide_points.loc[tide_points['Z'] > mean_sea_level, 'Z'] = mean_sea_level
tide_points["geometry"] = tide_points.apply(lambda row: shapely.geometry.Point([row["geometry"].x, row["geometry"].y, row["Z"]]), axis=1)

local_drive = pathlib.Path(r"c:\Local\data\DEMs\tide model")
tide_points.to_file(local_drive / "nzvd2016_clipped_tide_model_points_max_set_to_msl.gpkg")

In [ ]:
tiles.dissolve().buffer(20000).explore()